In [1]:
# 🔧 Gerekli kütüphaneleri yükle
!pip install gdown azure-eventhub tqdm pandas --quiet
#gdown google drivedan veri indirmek icin, tqdm progress bar

StatementMeta(, c893b995-d93a-4fcc-8e97-0f5d647c0be7, 3, Finished, Available, Finished)

In [2]:
# 📦 Modülleri içe aktar
from azure.eventhub import EventHubProducerClient, EventData
import pandas as pd
import json
import os
from tqdm import tqdm
import zipfile

StatementMeta(, c893b995-d93a-4fcc-8e97-0f5d647c0be7, 4, Finished, Available, Finished)

In [ ]:
# 📥 Google Drive'dan ZIP dosyasını indir
import gdown

file_id = ""
url = f""
output = "uber_data.zip"  # olarak lokal dizine indirildi

gdown.download(url, output, quiet=False)

print("✅ ZIP dosyası başarıyla indirildi:", output)

StatementMeta(, c893b995-d93a-4fcc-8e97-0f5d647c0be7, 5, Finished, Available, Finished)

Downloading...
From (original): https://drive.google.com/uc?id=1MBAN2V06VwTbxE4ADBVnTz8mm6cgrkDd
From (redirected): https://drive.google.com/uc?id=1MBAN2V06VwTbxE4ADBVnTz8mm6cgrkDd&confirm=t&uuid=f9f2367c-58b6-4342-a1af-6949a0f1f82c
To: /mnt/var/hadoop/tmp/nm-secondary-local-dir/usercache/trusted-service-user/appcache/application_1748699470335_0001/container_1748699470335_0001_01_000001/uber_data.zip
 99%|█████████▉| 1.78G/1.80G [00:07<00:00, 217MB/s]


In [4]:
# 📂 ZIP içeriğini klasöre çıkar
import zipfile
import os

zip_path = "uber_data.zip"
output_dir = "trip_data_files"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)  #zip icerigini yukaridaki hedefe cikardik

print("📁 ZIP açıldı. Klasör içeriği:")
print(os.listdir(output_dir))

StatementMeta(, c893b995-d93a-4fcc-8e97-0f5d647c0be7, 6, Finished, Available, Finished)

📁 ZIP açıldı. Klasör içeriği:
['trip_fare_12.csv', 'trip_fare_10.csv', 'trip_fare_9.csv', 'trip_data_10.csv', 'trip_data_9.csv', 'trip_data_12.csv']


In [ ]:
# 📦 Gerekli kütüphaneler
from azure.eventhub import EventHubProducerClient, EventData
import pandas as pd
import json
import time
from tqdm import tqdm
import os

# 🔐 Yeni Connection String (data-eventstream için)
connection_str = "Endpoint=s"

# 📁 ZIP’ten açılmış dosyaların bulunduğu klasör
folder = "trip_data_files"  # veya senin ZIP açtığın path neyse

# 🎯 Gönderilecek dosya
target_file = "trip_fare_12.csv"
file_path = os.path.join(folder, target_file)

# ✅ Satır sayısını hesapla (başlık satırı hariç)
print(f"📊 Satır sayılıyor: {target_file}")
with open(file_path, 'r', encoding='utf-8') as f:
    total_rows = sum(1 for line in f) - 1

print(f"\n🚀 Gönderim başlıyor: {target_file} ({total_rows} satır)")
progress = tqdm(total=total_rows, desc=target_file, unit="rows")

# 📨 Event gönderimi
producer = EventHubProducerClient.from_connection_string(conn_str=connection_str) # eventhub a baglanmak icin istemci

for chunk in pd.read_csv(file_path, chunksize=1000):  #veri binlik satirlar halinde okundu
    batch = producer.create_batch() # toplu veri gonderimi
    for _, row in chunk.iterrows():  #sistemin hatasiz ilerlemesi icin
        try:
            json_data = json.dumps(row.to_dict(), default=str)
            event = EventData(json_data)
            try:
                batch.add(event)
            except ValueError:
                producer.send_batch(batch)
                batch = producer.create_batch()
                batch.add(event)
        except:
            pass
        progress.update(1)

    if batch:
        producer.send_batch(batch)
    time.sleep(0.01)

producer.close()
progress.close()

print(f"\n✅ {target_file} başarıyla gönderildi.")


StatementMeta(, c893b995-d93a-4fcc-8e97-0f5d647c0be7, 12, Finished, Available, Finished)

📊 Satır sayılıyor: trip_fare_12.csv


trip_fare_12.csv: 100%|█████████▉| 13933001/13971118 [29:29<00:04, 8140.34rows/s]


In [1]:
!pip install azure-eventhub


StatementMeta(, 9909cd70-a57b-4f50-8c95-2547668386c5, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 16.7 MB/s eta 0:00:00


In [ ]:
from azure.eventhub import EventHubProducerClient, EventData
import json

# Event Hub bağlantı bilgisi
connection_str = (
    "Endpoint=sb://"
    "SharedAccessKeyName=key_"
    "SharedAccessKey="
    "EntityPath="
)

eventhub_name = ""

# 10.000 satır al
df_sample = spark.table("sample.sample_trip1").limit(10000)
json_rows = df_sample.toJSON().collect()

# Event Hub producer başlat
producer = EventHubProducerClient.from_connection_string(
    conn_str=connection_str,
    eventhub_name=eventhub_name
)

# Gönderim
batch = producer.create_batch()
for row in json_rows:
    try:
        batch.add(EventData(row))
    except ValueError:
        producer.send_batch(batch)
        batch = producer.create_batch()
        batch.add(EventData(row))

# Son batch'i de gönder
producer.send_batch(batch)
producer.close()

print("✅ İlk 10.000 satır başarıyla gönderildi.")


In [ ]:
from azure.eventhub import EventHubProducerClient, EventData
import json
from datetime import datetime, date

connection_str = (
    "Endpoint=sb://;"
    "SharedAccessKeyName=key"
    "SharedAccessKey="
    "EntityPath="
)

eventhub_name = "e"

# 10.001–10.500 arasındaki veriyi al
df_limit = spark.table("sample.sample_trip1").limit(13000)
rows = df_limit.tail(500)

# 🛠️ JSON'a uygun hale getir
def serialize_row(row):
    def convert(v):
        if isinstance(v, (datetime, date)):
            return v.isoformat()
        return v
    return {k: convert(v) for k, v in row.asDict().items()}

json_rows = [json.dumps(serialize_row(row)) for row in rows]

# EventHub gönderimi
producer = EventHubProducerClient.from_connection_string(
    conn_str=connection_str,
    eventhub_name=eventhub_name
)

batch = producer.create_batch()
for row in json_rows:
    try:
        batch.add(EventData(row))
    except ValueError:
        producer.send_batch(batch)
        batch = producer.create_batch()
        batch.add(EventData(row))

producer.send_batch(batch)
producer.close()

print("✅ 10.001–10.500 arası 500 kayıt başarıyla gönderildi.")


StatementMeta(, 9909cd70-a57b-4f50-8c95-2547668386c5, 4, Finished, Available, Finished)

✅ 10.001–10.500 arası 500 kayıt başarıyla gönderildi.


In [4]:
df_fare = spark.table("sample.sample_fare1")


StatementMeta(, afcf95e0-9df4-4f05-b7a9-8ce1593bbf95, 6, Finished, Available, Finished)

In [5]:
df_fare.printSchema()


StatementMeta(, afcf95e0-9df4-4f05-b7a9-8ce1593bbf95, 7, Finished, Available, Finished)

root
 |-- trip_id: string (nullable = true)
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- ingest_date: date (nullable = true)
 |-- is_tip_anomaly: boolean (nullable = true)
 |-- is_fare_outlier: boolean (nullable = true)
 |-- is_tip_outlier: boolean (nullable = true)
 |-- is_total_outlier: boolean (nullable = true)
 |-- month: integer (nullable = true)

